In [28]:
'''importing modules'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys
from time import sleep
sys.path.append('/home/mekasaimukund/miniconda3/envs/nn/lib/python3.10/site-packages')

import gym
import pygame
from collections import defaultdict

# params
global eps
eps = 0.1
alpha = 0.1
gamma = 0.6
train_epochs = int(1e5)
MAX_LIM = 200

pygame.init()
print('Imported modules')

Imported modules


In [29]:
# environment setup
env = gym.make('Taxi-v3').env
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [30]:
Q_table = np.zeros( [env.observation_space.n, env.action_space.n] )
frames = list()

In [31]:
def print_qtable(e):
    # for frame in frames:
        # print(frame)

    with open('qtable' + str(eps) + '.txt', 'w') as f:
        for i in range(len(Q_table)):
            for j in range(len(Q_table[i])):
                f.write(str(Q_table[i][j]) + ' ')
            f.write('\n')

    print('Time: {}'.format(e))


def take_action(state):
# epsilon greedy policy
    if random.random() < eps:
        return env.action_space.sample()
    else:
        return np.argmax(Q_table[state])
        

def update_Q_table(alpha, gamma, state, action, reward, next_state):
    Q_table[state, action] = Q_table[state, action] + alpha*( reward + gamma * np.max(Q_table[next_state]) - Q_table[state, action] ) 
    # Q_table[state, action] = (1 - alpha)*Q_table[state, action] + alpha*( reward + gamma * np.max(Q_table[next_state]) ) 


In [32]:
for _ in range(train_epochs):
    cur_state = env.reset()
    goal = False

    time = 0
    while not goal:
        action = take_action(cur_state)

        next_state, reward, goal, info = env.step(action)
        # print(next_state, reward, goal, info)

        update_Q_table(alpha, gamma, cur_state, action, reward, next_state)
        cur_state = next_state

        time += 1
    # print('Time: {}'.format(time))
    
print_qtable(train_epochs)
print('Training finished')

Time: 100000
Training finished


In [35]:
# Working of agent
runs = 100
mean_time = 0
penalties = 0
for r in range(runs):
    state = env.reset()
    done = False

    while done is False:
        action = np.argmax(Q_table[state, :]) # take_action(state)
        next_state, action, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        state = next_state  
        mean_time += 1

print('Mean time: {}'.format(mean_time/runs))
print('Mean Penalties: {}'.format(penalties/runs))

Mean time: 12.95
Mean Penalties: 0.0


In [36]:
old_table = np.zeros( (env.observation_space.n, env.action_space.n) )
with open ('qtable.txt', 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        line = line.strip()
        old_table[i] = np.array(line.split(' '))

diff = 0
for i in range(len(Q_table)):
    for j in range(len(Q_table[i])):
        diff = max(diff, Q_table[i][j] - old_table[i][j])
print(diff)

0.10872210151266337
